In [18]:
import pandas as pd
import json
from IPython.display import display
import os

# Atrybuty z OSM - złączenie USA i PL

In [3]:
ny = pd.read_csv("../data/downstream_tasks/airbnb/new_york/New York City_all_tags_one_hot_encoded.csv")
ny = ny.drop(["osmid", 'geometry'], axis=1)
ny

,aeroway_aerodrome,aeroway_apron,aeroway_gate,aeroway_hangar,aeroway_helipad,aeroway_heliport,aeroway_holding_position,aeroway_jet_bridge,aeroway_navigationaid,aeroway_parking_position,...,waterway_fuel,waterway_link,waterway_lock_gate,waterway_put_in,waterway_river,waterway_sluice_gate,waterway_stream,waterway_tidal_channel,waterway_waterfall,waterway_weir
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1381066,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1381067,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1381068,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1381069,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [4]:
st = pd.read_csv("../data/downstream_tasks/airbnb/seattle/Seattle_all_tags_one_hot_encoded.csv")
st = st.drop(["osmid", 'geometry'], axis=1)
st

,feature_id,aeroway_aerodrome,aeroway_apron,aeroway_hangar,aeroway_helipad,aeroway_holding_position,aeroway_navigationaid,aeroway_parking_position,aeroway_runway,aeroway_service,...,waterway_ditch,waterway_dock,waterway_drain,waterway_fish_pass,waterway_lock_gate,waterway_river,waterway_slough,waterway_stream,waterway_waterfall,waterway_yes
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266888,613,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
266889,614,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
266890,615,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
266891,616,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [ ]:
nycols = list(ny.columns)
nycols

In [6]:
len(nycols + list(st.columns))

1885

In [7]:
cities_pol = ["poznan", "wroclaw", "warszawa", "krakow", "szczecin"]
all_cols = []
for city in cities_pol:
    df = pd.read_parquet(f"../data/organized-datasets/{city}/2022/h9/count-embedder/dataset.parquet")
    df = df.drop(["region_id", "geometry"], axis=1)
    city_cols = df.columns
    all_cols.extend(city_cols)
    print(all_cols)

['aeroway_aerodrome', 'aeroway_apron', 'aeroway_gate', 'aeroway_hangar', 'aeroway_helipad', 'aeroway_holding_position', 'aeroway_parking_position', 'aeroway_runway', 'aeroway_stopway', 'aeroway_taxiway', 'aeroway_terminal', 'amenity_animal_shelter', 'amenity_animal_training', 'amenity_arts_centre', 'amenity_atm', 'amenity_baby_hatch', 'amenity_bank', 'amenity_bar', 'amenity_bbq', 'amenity_bench', 'amenity_bicycle_parking', 'amenity_bicycle_rental', 'amenity_bicycle_repair_station', 'amenity_biergarten', 'amenity_bo', 'amenity_boat_rental', 'amenity_boat_storage', 'amenity_bureau_de_change', 'amenity_bus_station', 'amenity_cafe', 'amenity_car_rental', 'amenity_car_wash', 'amenity_casino', 'amenity_charging_station', 'amenity_childcare', 'amenity_cinema', 'amenity_clinic', 'amenity_clock', 'amenity_college', 'amenity_community_centre', 'amenity_compressed_air', 'amenity_conference_centre', 'amenity_courthouse', 'amenity_coworking_space', 'amenity_crematorium', 'amenity_dancing_school', '

In [ ]:
all_cols.extend(nycols)
all_cols.extend(list(st.columns))
print(all_cols)

['aeroway_aerodrome', 'aeroway_apron', 'aeroway_gate', 'aeroway_hangar', 'aeroway_helipad', 'aeroway_holding_position', 'aeroway_parking_position', 'aeroway_runway', 'aeroway_stopway', 'aeroway_taxiway', 'aeroway_terminal', 'amenity_animal_shelter', 'amenity_animal_training', 'amenity_arts_centre', 'amenity_atm', 'amenity_baby_hatch', 'amenity_bank', 'amenity_bar', 'amenity_bbq', 'amenity_bench', 'amenity_bicycle_parking', 'amenity_bicycle_rental', 'amenity_bicycle_repair_station', 'amenity_biergarten', 'amenity_bo', 'amenity_boat_rental', 'amenity_boat_storage', 'amenity_bureau_de_change', 'amenity_bus_station', 'amenity_cafe', 'amenity_car_rental', 'amenity_car_wash', 'amenity_casino', 'amenity_charging_station', 'amenity_childcare', 'amenity_cinema', 'amenity_clinic', 'amenity_clock', 'amenity_college', 'amenity_community_centre', 'amenity_compressed_air', 'amenity_conference_centre', 'amenity_courthouse', 'amenity_coworking_space', 'amenity_crematorium', 'amenity_dancing_school', '

In [8]:
all_attrs = {}
all_attrs["cols"] = list(set(all_cols))
with open('../data/hex_attributes.json', 'w') as f:
    json.dump(all_attrs, f)

In [ ]:
with open('../data/hex_attributes.json', 'r') as file:
        all_attrs = json.load(file)
all_attrs['cols']

In [ ]:
all_columns = all_attrs['cols']
all_columns

In [11]:
len(all_columns)

1643

# Dodanie kolumn zgodnie z tagami

## Count embedder

In [ ]:
cities_pol = ["poznan", "wroclaw", "warszawa", "krakow", "szczecin"]
year=2022
for city in cities_pol:
    df = pd.read_parquet(f"../data/organized-datasets/{city}/2022/h9/count-embedder/dataset.parquet")
    for col in all_columns:
        if col not in df.columns:
            df[col] = 0
    display(df)
    if not os.path.exists(f"../data/organized-hexes/{city}/{year}/h9/count-embedder/"):
        os.makedirs(f"../data/organized-hexes/{city}/{year}/h9/count-embedder/")
    df.to_parquet(f"../data/organized-hexes/{city}/{year}/h9/count-embedder/dataset.parquet")


In [ ]:
cities_usa = ["New_York", "Seattle"]
year=2023
for city in cities_usa:
    df = pd.read_csv(f"../data/organized-datasets/organized_USA/{city}_res_9_features.csv")
    for col in all_columns:
        if col not in df.columns:
            df[col] = 0
    display(df)
    if not os.path.exists(f"../data/organized-hexes/{city}/{year}/h9/count-embedder/"):
        os.makedirs(f"../data/organized-hexes/{city}/{year}/h9/count-embedder/")
    df.to_parquet(f"../data/organized-hexes/{city}/{year}/h9/count-embedder/dataset.parquet")